In [ ]:
import os 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
train_path = '/kaggle/input/spaceship-titanic/train.csv'
test_path = '/kaggle/input/spaceship-titanic/test.csv'

In [ ]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
import missingno as msno 
_ = msno.matrix(df_train)

In [ ]:
df_train.info()

In [ ]:
df_numeric = df_train.select_dtypes(exclude=['object','bool'])
df_object_bool = df_train.select_dtypes(include=['object','bool'])

Fitting using regression model

In [ ]:
%%time
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(estimator=RandomForestRegressor(), random_state=0)
imp_mean.fit(df_numeric)

In [ ]:
df_numeric_imputed = pd.DataFrame(imp_mean.transform(df_numeric),columns=df_numeric.columns)

In [ ]:
df_train_imputed = pd.concat([df_numeric_imputed,df_object_bool],axis=1)

In [ ]:
df_train_imputed.isnull().sum()

In [ ]:
df_train_imputed.dropna(inplace=True)

In [ ]:
df_train_imputed.shape

In [ ]:
objectCol = [col for col in df_train_imputed.columns if df_train_imputed[col].dtype=='object' or df_train_imputed[col].dtype=='bool']

for col in objectCol:
    print(f"{col}:\t column is having {df_train_imputed[col].nunique()}:\t unique values")

In [ ]:
for col in objectCol:
    if df_train_imputed[col].nunique()<10:
        df_train_imputed[col] = df_train_imputed[col].astype('category')

In [ ]:
df_train_imputed.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
df_train_imputed.info()

In [ ]:
for col in df_train_imputed.select_dtypes(include=['category']):
        df_train_imputed[col] = encoder.fit_transform(df_train_imputed[col])
#df_train_imputed['Transported'] = encoder.fit_transform(df_train_imputed['Transported'])

In [ ]:
df_train_imputed.head()

In [ ]:
df_train_imputed.drop(['PassengerId','Cabin','Name'],axis=1,inplace=True)

In [ ]:
df_train_imputed.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_train_imputed[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])
df_train_imputed[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = scaler.transform(df_train_imputed[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])
#df_train = scaler.fit_transform(df_train)

In [ ]:
X = df_train_imputed.drop('Transported',axis=1)
y = df_train_imputed.Transported

In [ ]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.3)#,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(xtrain,ytrain)
print("Training score",lr_clf.score(xtrain,ytrain))
print("Testing score",lr_clf.score(xtest,ytest))


params = {'C':np.linspace(0.1,1,10)}
lr = GridSearchCV(lr_clf,params,cv=10)
lr.fit(xtrain,ytrain)


print(lr.best_params_)
print(lr.best_score_)

print(lr.score(xtest,ytest))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(lr, xtest, ytest)  
plt.show()

In [ ]:
from sklearn.svm import SVC

lr_svc = SVC() 
lr_svc = lr_svc.fit(xtrain,ytrain)
print("Training score",lr_svc.score(xtrain,ytrain))
print("Testing score",lr_svc.score(xtest,ytest))

In [ ]:
lr_svc.get_params()

Still working on it not yet complete

In [ ]:
from sklearn.ensemble import RandomForestClassifier  

In [ ]:
lr_rfc = RandomForestClassifier(n_estimators=100,criterion='gini',min_samples_split=20,oob_score=True,max_features=4,min_samples_leaf=2 )

In [ ]:
lr_rfc.get_params().keys()

In [ ]:
lr_rfc.fit(xtrain,ytrain)
lr_rfc.score(xtrain,ytrain)

In [ ]:
lr_rfc.oob_score_

In [ ]:
lr_rfc.score(xtest,ytest)

Test dataset

In [ ]:
df_numeric = df_test.select_dtypes(exclude=['object','bool'])
df_object_bool = df_test.select_dtypes(include=['object','bool'])

In [ ]:
df_numeric_imputed = pd.DataFrame(imp_mean.transform(df_numeric),columns=df_numeric.columns)
df_test_imputed = pd.concat([df_numeric_imputed,df_object_bool],axis=1)

In [ ]:
df_test_imputed.dropna(inplace=True)
objectCol = [col for col in df_test_imputed.columns if df_test_imputed[col].dtype=='object' or df_test_imputed[col].dtype=='bool']

for col in objectCol:
    print(f"{col}:\t column is having {df_test_imputed[col].nunique()}:\t unique values")
    
for col in objectCol:
    if df_test_imputed[col].nunique()<10:
        df_test_imputed[col] = df_test_imputed[col].astype('category')
        
        
for col in df_test_imputed.select_dtypes(include=['category']):
        df_test_imputed[col] = encoder.fit_transform(df_test_imputed[col])
df = df_test_imputed.drop(['PassengerId','Cabin','Name'],axis=1)


df[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = scaler.transform(df[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])

In [ ]:
predicted = lr_rfc.predict(df)

In [ ]:
result = []
for pred in predicted:
    if pred==1:
        result.append(True)
    else:
        result.append(False)

In [ ]:
result_df = pd.DataFrame({'PassengerId':df_test_imputed.PassengerId,'Transported':result})

In [ ]:
result_df.to_csv('submission.csv',index=False)

In [ ]:
pd.read_csv('submission.csv')